In [ ]:
from PIL import Image
import pytesseract
import cv2
import os
import shutil
from pyaspeller import Word
from pyaspeller import YandexSpeller

pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

## Картинку в текст и просмотр ошибок в %

In [ ]:
def pic_to_text(image):

    preprocess = "thresh"

#     загрузить образ и преобразовать его в оттенки серого
    image = cv2.imread(image)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # проверьте, следует ли применять пороговое значение для предварительной обработки изображения
    if preprocess == "thresh":
        gray = cv2.threshold(gray, 0, 255,
            cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]

    # если нужно медианное размытие, чтобы удалить шум
    elif preprocess == "blur":
        gray = cv2.medianBlur(gray, 3)

    # сохраним временную картинку в оттенках серого, чтобы можно было применить к ней OCR
    filename = "{}.png".format(os.getpid())
    cv2.imwrite(filename, gray)
    
    # Теперь применим OCR к изображению, используя pytesseract:
    # загрузка изображения в виде объекта image Pillow, применение OCR, а затем удаление временного файла
    text = pytesseract.image_to_string(Image.open(filename), lang='rus')
    os.remove(filename)
    return text

In [ ]:
def check_spec_symbols(test_str):
#     pattern = r'[^\.a-z0-9]'
    pattern = r'[^\.a-zA-Zа-яА-Я]'
    if re.search(pattern, test_str):
        #Character other then . a-z 0-9 was found
        return True
    else:
        #No character other then . a-z 0-9 was found
        return False

In [155]:
def get_clean_text(name):
    t = pic_to_text(name)
    
    speller = YandexSpeller()
    text = t
    spl = text.split()
    
    for el in spl:
        if (len(el) == 1 or check_spec_symbols(el)):
            spl.remove(el)
            
    count = 0
    for el in spl:
        check = Word(el)
        if (check.correct == False):
            count += 1
            # create df and fill in
        #     print('el: {}, correct: {}, count: {}'.format(el, check.correct, count))
    perc = count / len(spl) * 100
    print('{}% of mistakes!'.format(perc))

In [157]:
get_clean_text('bad.jpg')

41.66666666666667% of mistakes!


## Проверить разом текст и заменить слова, не всегда работает

In [159]:
def get_text(text):
    changes = {change['word']: change['s'][0] for change in speller.spell(text)}
    for word, suggestion in changes.items():
        text = text.replace(word, suggestion)
    print(text)

## Проверить варианты исправления и сохранить вариант

In [71]:
check.variants

['хина']

In [ ]:
check.spellsafe
u'test'